In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from data_loader import DataLoader
from engine import GammaScalping
# from dotenv import load_dotenv
import yfinance as yf

In [ ]:
# 加载 .env 文件
# load_dotenv()

# 参数设置
params = {
    "start_date": "2022-01-01",
    "end_date": "2025-04-24",
    "initial_capital": 10000000.0,
    "hedge_freq_days": 2
}

# df = pd.DataFrame({
#     "Date":  "2022-01-01",
#     "SpotPrice": spot_df["Price"],
#     "CallPrice": call_df["Price"],
#     "PutPrice": put_df["Price"],
#     "CallIV": iv_df["CallIV"],
#     "PutIV": iv_df["PutIV"],
#     "PerpPrice": perp_df["Price"]
# })

def download_btc_data_yahoo(start_date, end_date, file_path):
    spot_df = yf.download("BTC-USD", start=start_date, end=end_date)
    spot_df.reset_index(inplace=True)
    spot_df.to_csv(file_path, index=False)
    return spot_df

In [7]:
file_path = "data.csv"
if not os.path.exists(file_path):
    try:
        download_btc_data_yahoo(params["start_date"], params["end_date"], file_path)
    except Exception as e:
        raise Exception("无法从Yahoo Finance下载数据，请检查网络连接或稍后重试: " + str(e))

spot_df = pd.read_csv(file_path)
df = spot_df[["Date", "Close"]].rename(columns={"Close": "SpotPrice"})

# 模拟期权价格和IV
# np.random.seed(42)
# n = len(df)
# df["CallPrice"] = df["SpotPrice"] * (0.05 + 0.01 * np.random.randn(n))
# df["PutPrice"] = df["SpotPrice"] * (0.04 + 0.01 * np.random.randn(n))
# df["CallIV"] = 0.6 + 0.05 * np.random.randn(n)
# df["PutIV"] = 0.65 + 0.05 * np.random.randn(n)
# df["PerpPrice"] = df["SpotPrice"] * (1 + 0.001 * np.random.randn(n))

# 加载数据
loader = DataLoader(file_path)
# expiry = loader.get_first_friday(2025, 4)
# # 将到期日转换为datetime类型并计算剩余天数
# expiry = pd.to_datetime(expiry)
# # 将expiry转换为datetime.date类型以匹配计算
# # 确保expiry是datetime类型并计算剩余天数

df = loader.add_expiry_days()

df.to_csv(file_path, index=False)

IllegalMonthError: bad month number nan; must be 1-12

In [ ]:
# 策略初始化
engine = GammaScalping(df, params["initial_capital"], params["hedge_freq_days"])

results = []
for idx, row in df.iterrows():
    today = row['Date'].date()
    # 到期日建仓
    if today == row['Expiry']:
        if engine.current_position:
            engine.close_position(row)
        try:
            engine.open_position(row)
        except RuntimeError as e:
            print(f"建仓失败: {e}")
            continue
    # 对冲
    if engine.current_position and (engine.last_hedge_day is None or (today - engine.last_hedge_day).days >= params["hedge_freq_days"]):
        call_delta, put_delta, perp_delta, total_delta = engine.delta_hedging(row, today)
    else:
        call_delta = put_delta = perp_delta = total_delta = 0.0
    # 组合跟踪
    if engine.current_position:
        record = engine.track_portfolio(row, call_delta, put_delta, perp_delta, total_delta)
        if record:  # Ensure the record is not None or empty
            results.append(record)

if results:
    result_df = pd.DataFrame(results)
    display(result_df)
else:
    print("No results to display. Please check the strategy logic.")
    # Create a placeholder DataFrame with necessary columns to avoid KeyError
    result_df = pd.DataFrame(columns=['Date', 'TotalAsset'])

# Ensure result_df is not empty before plotting or calculating metrics
if not result_df.empty:
    # 净值曲线
    plt.figure(figsize=(12, 6))
    # 检查日期列名是否为 'date' 而不是 'Date'
    plt.plot(result_df.index if 'Date' not in result_df.columns else result_df['Date'], 
            result_df['TotalAsset'])
    plt.title("BTC Gamma Scalping 策略净值曲线")
    plt.xlabel("日期")
    plt.ylabel("总资产 (BTC)")
    plt.grid()
    plt.show()

    # 最大回撤
    roll_max = result_df['TotalAsset'].cummax()
    drawdown = (result_df['TotalAsset'] - roll_max) / roll_max
    max_drawdown = drawdown.min()
    print(f"最大回撤: {max_drawdown:.2%}")

    # 夏普比率
    returns = result_df['TotalAsset'].pct_change().dropna()
    sharpe = returns.mean() / returns.std() * (252 ** 0.5)
    print(f"夏普比率: {sharpe:.2f}")
else:
    print("Result DataFrame is empty. Skipping visualization and metrics calculation.")

# 净值曲线
plt.figure(figsize=(12, 6))
# 检查日期列名是否为 'date' 而不是 'Date'
plt.plot(result_df.index if 'Date' not in result_df.columns else result_df['Date'], 
        result_df['TotalAsset'])
plt.title("BTC Gamma Scalping 策略净值曲线")
plt.xlabel("日期")
plt.ylabel("总资产 (USD)")
plt.grid()
plt.show()

# 最大回撤
roll_max = result_df['TotalAsset'].cummax()
drawdown = (result_df['TotalAsset'] - roll_max) / roll_max
max_drawdown = drawdown.min()
print(f"最大回撤: {max_drawdown:.2%}")

# 夏普比率
returns = result_df['TotalAsset'].pct_change().dropna()
sharpe = returns.mean() / returns.std() * (252 ** 0.5)
print(f"夏普比率: {sharpe:.2f}")